# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import pickle
import numpy as np
import pandas as pd

import os

# Data

In [30]:
data = [["layer1", "layer2", "layer3", "target"]]

In [29]:
def getDividedArray(array):
  '''
  Breaks longer arrays into chunks of length four
  :param array: array of layer sequence
  :return array containing chunks of length four
  :rtype: array
  '''
  temp = []
  if len(array) > 4:
    for i in range(len(array)-3):
      temp.append(array[i:i+4])
  return temp


def unpickleData(filename):
  '''
  Get data from pickled files
  :param filename: pickle file with model layer sequence
  :param data: the array containing the model array
  '''
  fileObj = open(filename,'rb')
  array = pickle.load(fileObj)
  if len(array) > 4:
    array = getDividedArray(array)
    for i in array:
      data.append(i)
  else:
    data.append(array)
  fileObj.close()
  return data

In [32]:
for file in os.listdir('../data/models'):
    filename=os.path.join('../data/models',file)
    unpickleData(filename)
    break

In [33]:
print(data)

[['layer1', 'layer2', 'layer3', 'target'], ['Conv2D', 'Conv2D', 'Conv2D', 'MaxPooling2D'], ['Conv2D', 'Conv2D', 'MaxPooling2D', 'Flatten'], ['Conv2D', 'MaxPooling2D', 'Flatten', 'Dense'], ['MaxPooling2D', 'Flatten', 'Dense', 'Dense'], ['Flatten', 'Dense', 'Dense', 'Dense'], ['Dense', 'Dense', 'Dense', 'Dense']]


In [34]:
# Column Names
# data = [["layer1", "layer2", "layer3", "target"]]

# get all data in data array
# connect to ec2 and call unpickleData


# convert it to csv
np.savetxt("../data/csv/data.csv", data, delimiter=",", fmt='%s')

In [35]:
df=pd.read_csv("../data/csv/data.csv")
df.head()

,layer1,layer2,layer3,target
0,Conv2D,Conv2D,Conv2D,MaxPooling2D
1,Conv2D,Conv2D,MaxPooling2D,Flatten
2,Conv2D,MaxPooling2D,Flatten,Dense
3,MaxPooling2D,Flatten,Dense,Dense
4,Flatten,Dense,Dense,Dense


# Model

Refrenece for the model
[link](https://stackoverflow.com/questions/64209325/if-i-want-to-predict-the-next-element-in-a-sequence-of-numbers-what-do-i-need-t)

For survey:
- [Making Predictions with Sequences](https://machinelearningmastery.com/sequence-prediction/)
- 

In [ ]:
model = Sequential()
model.add(layers.LSTM(units = 4, return_sequences = True))
model.add(layers.LSTM(units = 2))
model.add(layers.Dense(units = 1))
model.compile(optimizer = 'rmsprop', loss = 'mse')

# Prediction and analysis